In [140]:
from ultralytics import YOLO
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
import skimage
from PIL import Image
from utils import *

import importlib
from sklearn.metrics import mean_squared_error

import json
import yaml
import time
import supervision as sv
from inference import get_model
print(sv.__version__)
import os

0.25.0


In [184]:
src="./testvid.mp4"
target="./out)livars.mp4"

In [185]:
model = get_model(
    model_id="football-ball-detection-rejhg/4", 
    api_key="fpgo7lotAA2MTZARBCtt"
)


In [ ]:
import centroid_track
import utils
importlib.reload(centroid_track)
importlib.reload(utils)
# Specify the path to the video file
video_path = src

# Open the video file
cap = cv2.VideoCapture(video_path)

# Check if the video was successfully opened
if not cap.isOpened():
    print("Error: Unable to open video file.")
else:
# Read the first frame
    pass

# Get video properties for output writer
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object
output_path = 'output_city.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # You can use other codecs like 'MP4V' for MP4 files
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))



slice_dimensions = (640, 480)  # Width and height for slicing
overlap_dimensions = (100, 100)  # Overlap between slices
slice_wh=slice_dimensions
overlap_wh=overlap_dimensions
f_no=30
frame_number=0

# Initialize the BallTracker
tracker = centroid_track.BallTracker(buffer_size=5)
while True:
    ret, frame = cap.read()

    frame_number+=1

    #if frame_number==f_no:
        # Convert the frame from BGR to RGB
        
    patches = slice_frame(frame, slice_wh, overlap_wh)
    
    
    
    print(frame_number)
        # Step 2: Run inference on patches
    detection_frame,det_ball = infer_on_patches(model, patches, frame_number,0.6,0.4)
    print("ball detection:  ",det_ball)
    print(" detection frame length is:  ",len(detection_frame))
    if len(detection_frame)>1:
        print("===============================================================================Multuple DETECTION-------------------------------")
    
  

    detections_track = []

    if isinstance(detection_frame, np.ndarray) and detection_frame.size > 0:  # Check if it's a NumPy array with elements
        print("inside the FIRST instance-----111")
        x1, y1, x2, y2, conf = detection_frame[0]
        x_centroid=(x1 + x2) // 2
        y_centroid=(y1 + y2) // 2
        print("the centroid for tracking in FIRST instance are: ",x_centroid,y_centroid)
        detections_track.append([x_centroid,y_centroid ])  # Append the centroid
        detections_track = np.array(detections_track)
        tracked_ball = tracker.update(detections_track)
    elif isinstance(detection_frame, list) and len(detection_frame) > 0:  # Check if it's a non-empty list
        print("inside the SECOND instance-----111")
        x1, y1, x2, y2, conf = detection_frame[0]
        x_centroid=(x1 + x2) // 2
        y_centroid=(y1 + y2) // 2
        print("the centroid for tracking in SECOND instance are: ",x_centroid,y_centroid)
        detections_track.append([x_centroid,y_centroid // 2])  # Append the centroid
        detections_track = np.array(detections_track)
        tracked_ball = tracker.update(detections_track)
    else:
        print("inside none tracker update-------")
        tracked_ball = tracker.update(None)  
            
    
    # Draw final detections
    
    
    x1=int(x1)
    x2=int(x2)
    y1=int(y1)
    y2=int(y2)
    print(x1,y1,x2,y2)
    color = (0, 255, 0)
    thickness = 2
    cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)


    # Highlight the tracked ball
    if tracked_ball is not None:
        cx, cy = int(tracked_ball[0]), int(tracked_ball[1])
        radius = 10  # Set the desired radius for the circle
        cv2.circle(frame, (cx, cy), radius, (0, 0, 255), 2) 
    # Step 3: Reconstruct the frame
    #reconstructed_frame = reconstruct_frame(processed_patches, frame.shape)


    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    out.write(frame)
    # if det_ball==True:
    #     # Display the frame using matplotlib
    #     plt.figure(figsize=(10, 6))
    #     plt.imshow(frame_rgb)
    #     plt.axis("off")  # Turn off axes for better visualization
        
    #     plt.show()
    if frame_number==400:
        break
            
    


# Release the video capture object
cap.release()

1
Initial length of patches is:  2
x offset, y_offset ,patch number is:  0 0 0
Patch number with high confidence is:  0
Patch number with high confidence and it's offset is:  0 0
local x1,y1,x2,y2:  418 132 424 138
global x1,y1,x2,y2:  418 132 424 138
x offset, y_offset ,patch number is:  540 0 1
ball detection:   True
 detection frame length is:   1
inside the FIRST instance-----111
the centroid for tracking in FIRST instance are:  421.0 135.0
the detection inside the balltracker:  [[        421         135]]
The length of buffer is:  0
the buffer values are:  deque([], maxlen=5)
the distances are:  [          0]
the index are:  0
418 132 424 138
2
Initial length of patches is:  2
x offset, y_offset ,patch number is:  0 0 0
Patch number with high confidence is:  0
Patch number with high confidence and it's offset is:  0 0
local x1,y1,x2,y2:  418 132 424 138
global x1,y1,x2,y2:  418 132 424 138
x offset, y_offset ,patch number is:  540 0 1
ball detection:   True
 detection frame length

In [170]:
detection_frame=[[        422,         128,         428,         134 ]]
current_cent=[[425.0, 131.0]]
dq=[np.array([[        421,         135]]), np.array([[        422,         134]]), np.array([[        422,         134]]), np.array([[        423,         133]]), np.array([[        423,         132]])]

In [171]:
dq_concat=np.concatenate(dq)
print(dq_concat)
print(dq_concat.shape)
avg_cntroid = np.mean(dq_concat, axis=0)
print("the centroid is: ",avg_cntroid)

[[421 135]
 [422 134]
 [422 134]
 [423 133]
 [423 132]]
(5, 2)
the centroid is:  [      422.2       133.6]


In [172]:
current_cent

[[425.0, 131.0]]

In [173]:

cent_current_arr=np.array(current_cent).reshape(1,2)
cent_current_arr

array([[        425,         131]])

In [176]:
distances = np.linalg.norm(cent_current_arr - avg_cntroid, axis=1)
print(distances)

[      3.821]


In [177]:
index = np.argmin(distances)
index

0

In [178]:
current_cent[index]

[425.0, 131.0]

In [180]:
dq.append(cent_current_arr[index].reshape(1, -1))
dq

[array([[421, 135]]),
 array([[422, 134]]),
 array([[422, 134]]),
 array([[423, 133]]),
 array([[423, 132]]),
 array([[        425,         131]])]